# Preprocessing

In [2]:
# import models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeClassifier
from sklearn import neighbors
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
# import other libraries
import numpy as np
from sklearn.model_selection import cross_val_score
import csv
import pandas as pd
import os

In [3]:
# load data
class data_frame():
    def __init__(self, folder_path):
        self.X_train = pd.read_csv(os.path.join(folder_path, 'train/samples.csv')).drop('Unnamed: 0', axis=1).drop('subject', axis=1)
        self.y_train = pd.read_csv(os.path.join(folder_path, 'train/labels.csv')).drop('Unnamed: 0', axis=1)
        self.X_test = pd.read_csv(os.path.join(folder_path, 'test/samples.csv')).drop('Unnamed: 0', axis=1).drop('subject', axis=1)
        self.y_test = pd.read_csv(os.path.join(folder_path, 'test/labels.csv')).drop('Unnamed: 0', axis=1)

k=10
def load_data(src):
    data = []
    for i in range(0, k):
        data.append(data_frame(os.path.join(src, str(i))))
    
    return data
data = load_data("L:\\Autonomic Medicine\\Dysmorphology Photos\\Facial recognition project\\cchs-prediction\\data\\stratify_people\\train")

In [7]:
data

In [28]:
# create csv
OUTPUT_FILE = r"L:/Autonomic Medicine/Dysmorphology Photos/Facial recognition project/cchs-prediction/data/grid-search/grid_search_results2.csv"

# Grid Search

### Decision Tree

In [5]:
## Hyperparameter grid
criterion = ['gini', 'entropy']
max_depth = [3, 4, 5]
max_features = [55, 56]
max_leaf_nodes = list(range(2, 30))
min_impurity_decrease = [0, 0.1]
min_samples_leaf = [2, 3]

hyperparameters = [criterion, max_depth, max_features, max_leaf_nodes, min_impurity_decrease, min_samples_leaf]

In [6]:
class evaluation():
    def __init__(self, cnf_matrix):
        self.sensitivity = self.sensitivity(cnf_matrix)
        self.specificity = self.specificity(cnf_matrix)
    def sensitivity(self, cnf_matrix):
        tp = cnf_matrix[1][1]
        fn = cnf_matrix[1][0]
        return tp / (tp+fn)
    def specificity(self, cnf_matrix):
        tn = cnf_matrix[0][0]
        fp = cnf_matrix[0][1]
        return tn / (tn + fp)
    def __str__(self):
        return "Sensitivity: "+str(self.sensitivity)+" Specificity: "+str(self.specificity)

def evaluate(cnf):
    return evaluation(cnf)

In [41]:
def k_fold_prediction(m, model_name, data):
    cnf_mtx = 0 # model_name:sum(cnf_matrix) of k trials
    for df in data:
        m.fit(df.X_train, df.y_train.values.ravel())
        y_pred = m.predict(df.X_test)
        cnf_matrix = confusion_matrix(df.y_test, y_pred, labels=[0,1])

        cnf_mtx += cnf_matrix
    return cnf_mtx

In [17]:
import os
os.getcwd()

'l:\\Autonomic Medicine\\Dysmorphology Photos\\Facial recognition project\\cchs-prediction\\grid-search'

In [42]:
from sklearn.metrics import confusion_matrix
best_acc = -1
features = []

with open(OUTPUT_FILE, "a", newline='') as csvfile:
    csvWriter = csv.writer(csvfile)
    header = ["DecisionTreeClassifier", "Sensitivity", "Specificity", "features"]
    header.append(hyperparameters)
    csvWriter.writerow(header)

    for c in criterion:
        for d in max_depth:
            for f in max_features:
                for l in max_leaf_nodes:
                    for i in min_impurity_decrease:
                        for ml in min_samples_leaf:
                            model = DecisionTreeClassifier(criterion=c, max_depth=d, max_features=f, max_leaf_nodes=l, min_impurity_decrease=i, min_samples_leaf=ml)
                            model_name = "Decision Tree"
                            cnf_matrix = k_fold_prediction(model, model_name, data)
                            evalu = evaluation(cnf_matrix)
                            features = [c, d, f, l, i, ml]
                            csvWriter.writerow([model, evalu.sensitivity, evalu.specificity, features])
                            print(model, evalu.sensitivity, evalu.specificity, features)


DecisionTreeClassifier(max_depth=3, max_features=55, max_leaf_nodes=2,
                       min_impurity_decrease=0, min_samples_leaf=2) 0.0 1.0 ['gini', 3, 55, 2, 0, 2]
DecisionTreeClassifier(max_depth=3, max_features=55, max_leaf_nodes=2,
                       min_impurity_decrease=0, min_samples_leaf=3) 0.0 1.0 ['gini', 3, 55, 2, 0, 3]
DecisionTreeClassifier(max_depth=3, max_features=55, max_leaf_nodes=2,
                       min_impurity_decrease=0.1, min_samples_leaf=2) 0.0 1.0 ['gini', 3, 55, 2, 0.1, 2]
DecisionTreeClassifier(max_depth=3, max_features=55, max_leaf_nodes=2,
                       min_impurity_decrease=0.1, min_samples_leaf=3) 0.0 1.0 ['gini', 3, 55, 2, 0.1, 3]
DecisionTreeClassifier(max_depth=3, max_features=55, max_leaf_nodes=3,
                       min_impurity_decrease=0, min_samples_leaf=2) 0.14545454545454545 0.9489172681843421 ['gini', 3, 55, 3, 0, 2]
DecisionTreeClassifier(max_depth=3, max_features=55, max_leaf_nodes=3,
                       min_imp

### Logistic Regression

In [43]:
## Hyperparameter grid
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
penalty = ['l1', 'l2', 'elasticnet', 'none']
solver = ['saga', 'sag', 'liblinear', 'lbfgs', 'newton-cg']

hyperparameters = [C, penalty, solver]

In [45]:
## Initialize model, variables
best_acc = -1
features = []
with open(OUTPUT_FILE, "a", newline='') as csvfile:
    csvWriter = csv.writer(csvfile)
    header = ["Logistic Regression", "Sensitivity", "Specificity", "features"]
    header.append(hyperparameters)
    csvWriter.writerow(header)

    for c in C:
        for p in penalty:
            for s in solver:
                try:
                    model = LogisticRegression(C=c, penalty=p, solver=s)
                    model_name = "Logistic Regression"
                    cnf_matrix = k_fold_prediction(model, model_name, data)
                    evalu = evaluation(cnf_matrix)
                    features = [c, d, f, l, i, ml]
                    csvWriter.writerow([model, evalu.sensitivity, evalu.specificity, features])
                    print(model, evalu.sensitivity, evalu.specificity, features)
                except (ValueError):
                    continue

LogisticRegression(C=0.001, penalty='l1', solver='saga') 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]
LogisticRegression(C=0.001, penalty='l1', solver='liblinear') 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]
LogisticRegression(C=0.001, solver='saga') 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]
LogisticRegression(C=0.001, solver='sag') 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]
LogisticRegression(C=0.001, solver='liblinear') 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]
LogisticRegression(C=0.001) 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]
LogisticRegression(C=0.001, solver='newton-cg') 0.0 1.0 [0.001, 5, 56, 29, 0.1, 3]


C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting p

LogisticRegression(C=0.001, penalty='none', solver='saga') 0.3338842975206612 0.9583564686285397 [0.001, 5, 56, 29, 0.1, 3]


C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_

LogisticRegression(C=0.001, penalty='none', solver='sag') 0.4380165289256198 0.9455857856746253 [0.001, 5, 56, 29, 0.1, 3]


...

LogisticRegression(C=1000, penalty='none', solver='sag') 0.43636363636363634 0.9461410327595781 [1000, 5, 56, 29, 0.1, 3]


C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the da

LogisticRegression(C=1000, penalty='none') 0.5239669421487604 0.9244863964464186 [1000, 5, 56, 29, 0.1, 3]


C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\anmittal\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Us

LogisticRegression(C=1000, penalty='none', solver='newton-cg') 0.6859504132231405 0.9183786785119378 [1000, 5, 56, 29, 0.1, 3]


### Ridge Regression

In [46]:
## Hyperparameter grid
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 2, 4, 10]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']

hyperparameters = [alpha, solver]

In [47]:
## Initialize model, variables
best_acc = -1
features = []
with open(OUTPUT_FILE, "a", newline='') as csvfile:
    csvWriter = csv.writer(csvfile)
    header = ["Ridge Regression", "Sensitivity", "Specificity", "features"]
    header.append(hyperparameters)
    csvWriter.writerow(header)

    for a in alpha:
        for s in solver:
            try:
                model = RidgeClassifier(alpha=a, solver=s)
                model_name = "Ridge Regression"
                cnf_matrix = k_fold_prediction(model, model_name, data)
                evalu = evaluation(cnf_matrix)
                features = [c, d, f, l, i, ml]
                csvWriter.writerow([model, evalu.sensitivity, evalu.specificity, features])
                print(model, evalu.sensitivity, evalu.specificity, features)
            except (ValueError):
                continue

RidgeClassifier(alpha=0.0001) 0.6380165289256199 0.9400333148250972 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.0001, solver='svd') 0.6380165289256199 0.9400333148250972 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.0001, solver='cholesky') 0.6380165289256199 0.9400333148250972 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.0001, solver='lsqr') 0.5702479338842975 0.9444752915047196 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.0001, solver='sparse_cg') 0.6380165289256199 0.9394780677401444 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.0001, solver='sag') 0.5917355371900826 0.9561354802887285 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.0001, solver='saga') 0.5619834710743802 0.9572459744586341 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.001) 0.628099173553719 0.943364797334814 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.001, solver='svd') 0.628099173553719 0.943364797334814 [1000, 5, 56, 29, 0.1, 3]
RidgeClassifier(alpha=0.001, solver='cholesky') 0

### SVM

In [48]:
## Hyperparameter grid
degree = [0, 1, 2, 3, 4, 5, 6]
C = [0.0001, 100]
gamma = [0.0001, 0.001, 0.01, 0.1, 1, 2, 4, 10, 100]
kernel = ['rbf', 'linear']

hyperparameters = [degree, C, gamma, kernel]

In [49]:
best_acc = -1
features = []

with open(OUTPUT_FILE, "a", newline='') as csvfile:
    csvWriter = csv.writer(csvfile)
    header = ["SVM", "Sensitivity", "Specificity", "features"]
    header.append(hyperparameters)
    csvWriter.writerow(header)

    for d in degree:
        for c in C:
            for g in gamma:
                for k in kernel:
                    try:
                        model = svm.SVC(degree=d,C=c,gamma=g,kernel=k)
                        model_name = "SVM"
                        cnf_matrix = k_fold_prediction(model, model_name, data)
                        evalu = evaluation(cnf_matrix)
                        features = [c, d, f, l, i, ml]
                        csvWriter.writerow([model, evalu.sensitivity, evalu.specificity, features])
                        print(model, evalu.sensitivity, evalu.specificity, features)
                    except (ValueError):
                        continue

SVC(C=0.0001, degree=0, gamma=0.0001) 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.0001, kernel='linear') 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.001) 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.001, kernel='linear') 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.01) 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.01, kernel='linear') 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.1) 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=0.1, kernel='linear') 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=1) 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=1, kernel='linear') 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=2) 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=2, kernel='linear') 0.0 1.0 [0.0001, 0, 56, 29, 0.1, 3]
SVC(C=0.0001, degree=0, gamma=4) 0.0 1.0 [

### K-Nearest Neighbors

In [1]:
## Hyperparameter grid
leaf_size = list(range(1, 10))
n_neighbors = list(range(20, 40))
p = [1, 2, 3]

hyperparameters = [leaf_size, n_neighbors, p]

In [6]:
best_acc = -1
features = []

with open(OUTPUT_FILE, "a", newline='') as csvfile:
    csvWriter = csv.writer(csvfile)
    header = ["K-Nearest Neighbors", "Sensitivity", "Specificity", "features"]
    header.append(hyperparameters)
    csvWriter.writerow(header)

    for ls in leaf_size:
        for n in n_neighbors:
            for p_iter in p:
                try:
                    model = neighbors.KNeighborsClassifier(leaf_size=ls,n_neighbors=n,p=p_iter)
                    model_name = "K-Nearest Neighbors"
                    cnf_matrix = k_fold_prediction(model, model_name, data)
                    evalu = evaluation(cnf_matrix)
                    features = [c, d, f, l, i, ml]
                    csvWriter.writerow([model, evalu.sensitivity, evalu.specificity, features])
                    print(model, evalu.sensitivity, evalu.specificity, features)
                except (ValueError):
                    continue

### Naive Bayes

In [5]:
## Hyperparameter grid
var_smoothing = np.logspace(0, -9, num=100)

hyperparameters = [var_smoothing]

In [6]:
best_acc = -1
features = []

with open(OUTPUT_FILE, "a", newline='') as csvfile:
    csvWriter = csv.writer(csvfile)
    header = ["Naive Bayes", "Accuracy"]
    header.append(hyperparameters)
    csvWriter.writerow(header)

    for vs in var_smoothing:
        model = GaussianNB(var_smoothing=vs)
        scores = cross_val_score(model, X, y, cv=100, n_jobs=-1) # 100-fold cross-validation
        avg_accuracy = sum(scores) / len(scores) # evaluation metric

        # update best accuracy score
        if avg_accuracy > best_acc: 
            best_acc = avg_accuracy
            features = [vs]
            print(best_acc, features)
            row = ["Naive Bayes", best_acc]
            row.append(features)
            csvWriter.writerow(row)

0.7337931034482759 [1.0]
